In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
from torchvision import models,transforms
from torchvision.utils import make_grid
from torchvision.datasets import CIFAR10
from torch.utils.data.sampler import SubsetRandomSampler
from torch.utils.tensorboard import SummaryWriter
from torchsummary import summary
from torch.utils.data import Dataset, DataLoader


In [3]:
def cifar10(batch_sz, path='./datasets'):
    num_classes = 10
    transform_train = transforms.Compose([
                        transforms.RandomCrop(32, padding=4), # Increase the number of samples by moving the images around 
                        transforms.RandomHorizontalFlip(),
                        transforms.ToTensor(),
                    ])
    transform_test = transforms.Compose([
                        transforms.ToTensor(),
                    ])

    # Training dataset
    train_data = CIFAR10(root=path, train=True, download=True, transform=transform_train) # 
    train_loader = torch.utils.data.DataLoader(train_data, batch_size=batch_sz,
                                               shuffle=True, pin_memory=True)

    # Test dataset
    test_data = CIFAR10(root=path, train=False, download=True, transform=transform_test)
    test_loader = torch.utils.data.DataLoader(test_data,
                                              batch_size=batch_sz, shuffle=False, pin_memory=True)

    return train_loader, test_loader, num_classes

In [4]:
class Lecture11CNN(nn.Module):
    # Initialise / Constructor
    def __init__(self) -> None:
        super().__init__()
        self.conv1 = nn.Conv2d(3,10,3,1,1)

        self.bn1 = nn.BatchNorm2d(10) # has to be the same as rh

        self.conv2 = nn.Conv2d(64,128,3,1,1)
        self.conv3 = nn.Conv2d(128,265,3,1,1)
        self.conv4 = nn.Conv2d(256,128,3,1,1)
        self.conv5 = nn.Conv2d(128,64,3,1,1)

        self.maxpool = nn.MaxPool2d((2,2))
        self.leaky_relu = nn.LeakyReLU()

        self.fc6 = nn.Linear(64*4*4,100)
        self.fc7 = nn.Linear(100,10)


    def forward(self, x):
        x = self.leaky_relu(self.conv1(x))
        x = self.maxpool(x)
        x = self.leaky_relu(self.conv2(x))
        x = self.maxpool(x)
        x = self.leaky_relu(self.conv3(x))
        x = self.maxpool(x)
        x = self.leaky_relu(self.conv4(x))
        x = self.maxpool(x)
        x = self.leaky_relu(self.conv5(x))
        x = self.maxpool(x)

        x = x.view(-1, 4*4*64)

        x = self.fc6(x)
        x = self.leaky_relu(x)
        x = self.fc7(x)

        return x




In [ ]:
# Initialize Model

model_2 = CNN().to(device=device)



# Training
# Loss and Optimizer

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model_2.parameters(), lr=lr, momentum= mm)

writer = SummaryWriter(f'runs/MNIST/CNN_1_64_1e-2')

total_loss = []

step = 0

for epoch in range(epoch_no):
    train_loss = 0
    for batch_idx, (data, targets) in enumerate(train_loader):
        data = data.to(device=device)
        targets = targets.to(device=device)




        # forwards
        logits = model_2(data)
        loss = criterion(logits, targets)
        train_loss += loss.item()


        # backward 
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        # Running Accuracy

        _, predictions = logits.max(1)
        num_corr = (predictions == targets).sum()
        running_acc = float(num_corr)/float(data.shape[0])
        
        writer.add_scalar("Training Loss", loss, global_step = step) 
        writer.add_scalar("Accuracy", running_acc, global_step=step)
        step += 1  


    total_loss.append(train_loss)

In [ ]:
# Saliency 
# which parts of the image activate the network the most 

X = batch[0].to(device)

X = X[3,:,:,:]

X.requires_grad_